$ \frac{d\dot{m}}{dt} = \frac{A_1}{L_C}(\phi (N(t), \dot{m})P_1 - P_P(t)) $

$ \frac{d P_P}{dt} = \frac{C_1^2}{\nu _P}(\dot{m}(t) - \alpha (t) K_\nu \sqrt{P_P - P_{out}}) $

$ \begin{matrix} A_1 & = & 2.6\centerdot 10^-3 m² \\
\nu _P & = & 2.0 m³ \\
L_C & = & 2.0 m \\
K_\nu & = & \frac{0.38 kg}{(kPa)^{0.5}s} \\
P_1 & = & 4.5 Bar \\
P{out} & = & 5.0 Bar \end{matrix}
$

$ \frac{d\dot{m}}{dt} = \frac{2.6\centerdot 10^{-3}}{2.0}(1.5\centerdot 4.5 - P_P) $

$ \frac{d P_P}{dt} = \frac{479.029^2}{2.0}(\dot{m} - \alpha {0.38} \sqrt{P_P - 5.0}) $

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from scipy.optimize import fsolve
import casadi as ca
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [8]:
np.random.seed(42)
A1 = (2.6)*(10**-3)
Lc = 2
kv = 0.38
P1 = 4.5
P_out = 5
C = 479

timestep = 5
epochs = 500
units = 100
nData = 250
nAlphas = 10

alphas = np.random.uniform(0.35,0.65, nAlphas)

def fun(variables, A1, Lc, kv, P1, P_out, C) :
    (x,y) = variables
    eqn_1 = (A1/Lc)* ((1.5 * P1) - y)
    eqn_2 = (C**2)/2 * (x - alphas[0] * kv * np.sqrt(y - P_out))
    return [eqn_1, eqn_2]

result = fsolve(fun, (0, 10), args = (A1, Lc, kv, P1, P_out, C)) 

init_m = result[0]
init_p = result[1]
interval = [np.linspace(i * nData, (i + 1) * nData, nData) for i in range(nAlphas)]
x = ca.MX.sym('x', 2)
alpha = ca.MX.sym('alpha', 1)
massFlowrate = []
PlenumPressure = []
alpha_values = []
RNN_train = []
RNN_trainFut = []
RNN_test = []
nAlphas_Met = int(nAlphas/2)

for i in range(0,nAlphas): 
    alpha_values.append([np.full(nData, alphas[i])])

    rhs = ca.vertcat((A1/Lc)*((1.5 * P1) - x[1]), (C**2)/2 * (x[0] - alpha * kv * np.sqrt(x[1] - P_out)))
    ode = {'x' : x, 'ode' : rhs, 'p' : alpha }

    F = ca.integrator('F','idas', ode, interval[i][0], interval[i])
    
    sol = F(x0 = [init_m, init_p], p = alphas[i])

    xf_values = np.array(sol["xf"])

    aux1, aux2 = xf_values
    massFlowrate.append(aux1)
    PlenumPressure.append(aux2)
    init_m = aux1[-1]
    init_p = aux2[-1]

    if i < nAlphas_Met:
        RNN_train.append([aux1, aux2, np.full(nData,alphas[i])])
        RNN_trainFut.append([aux1, aux2, np.full(nData,alphas[i+1])])
    else:
        RNN_test.append([aux1, aux2, np.full(nData,alphas[i])])


In [9]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time", "Alpha vs Time"))

# Adiciona os gráficos de mass flow rate
for i in range(0, nAlphas):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines',
                             name='Mass Flow Rate', legendgroup='massflow', showlegend=i == 0), row = 1, col = 1)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines',
                             name='Plenum Pressure', legendgroup='pressure', showlegend=i == 0), row = 1, col = 2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(alpha_values[i]), mode='lines', 
                             name='Alpha Values', line=dict(dash='dash'), legendgroup='alpha', showlegend=i == 0), row = 1, col = 3)

# Atualiza layout
fig.update_layout(
    title='Mass Flow Rate vs Time',
    xaxis_title='Time',
    yaxis_title='Value',
    grid=dict(rows=1, columns=3),
    template='plotly',
)

# Mostra a figura
fig.show()


In [10]:
RNN_train = np.array(RNN_train)
RNN_trainFut = np.array(RNN_trainFut)
RNN_test = np.array(RNN_test)

X_train = []
y_trainM = []
y_trainP = []
x_test = []

for j in range(nAlphas_Met):
    for i in range(len(RNN_train[0][0])):
        X_train.append(RNN_train[j,:,i])
        x_test.append(RNN_test[j,:,i])
        if i == (len(RNN_train[0][0]) - 1):
            y_trainM.append(RNN_trainFut[j, :1, 0])
            y_trainP.append(RNN_trainFut[j, 1:2, 0])
        else:
            y_trainM.append(RNN_train[j,:1,i+1])
            y_trainP.append(RNN_train[j,1:2,i+1])

X_train = tf.reshape(X_train, [nData,nAlphas_Met,3])
y_trainM = tf.reshape(y_trainM, [nData,nAlphas_Met,1])
y_trainP = tf.reshape(y_trainP, [nData,nAlphas_Met,1])
x_test = tf.reshape(x_test, [nData,nAlphas_Met,3])

X_train = np.array(X_train)
#X_train = X_train/np.max(X_train)

y_trainM = np.array(y_trainM)
#y_trainM = y_trainM/np.max(y_trainM)

y_trainP = np.array(y_trainP)
#y_trainP = y_trainP/np.max(y_trainP)

x_test = np.array(x_test)
#x_testN = x_test/np.max(x_test)


In [11]:
# Definindo a entrada
input_layer = Input(shape=(timestep, 3))  # 3 variáveis de entrada

# Camada RNN
rnn_output = LSTM(units, return_sequences=True)(input_layer)

# Saídas para cada variável de saída
output1 = Dense(1,activation='relu', name='Mass')(rnn_output)  # Primeira saída
output2 = Dense(1,activation='relu', name='Press')(rnn_output)  # Segunda saída

# Modelo
model = Model(inputs=input_layer, outputs=[output1, output2])

model.compile(optimizer='adam', 
              loss={'Mass': 'mse', 
                    'Press': 'mse'},
              metrics={'Mass': 'mse', 
                       'Press': 'mse'})

history = model.fit(X_train, [y_trainM, y_trainP], epochs=epochs, verbose = 1)


Epoch 1/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - Mass_loss: 0.0686 - Mass_mse: 0.0685 - Press_loss: 39.9605 - Press_mse: 39.9935 - loss: 40.0621  
Epoch 2/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Mass_loss: 0.0504 - Mass_mse: 0.0505 - Press_loss: 26.7790 - Press_mse: 26.8022 - loss: 26.8527 
Epoch 3/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Mass_loss: 0.0111 - Mass_mse: 0.0110 - Press_loss: 17.0037 - Press_mse: 17.0180 - loss: 17.0291 
Epoch 4/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - Mass_loss: 0.0177 - Mass_mse: 0.0178 - Press_loss: 11.2577 - Press_mse: 11.2652 - loss: 11.2830 
Epoch 5/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Mass_loss: 0.0071 - Mass_mse: 0.0071 - Press_loss: 8.6418 - Press_mse: 8.6439 - loss: 8.6510 
Epoch 6/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Mass_loss: 0.0044 - Mass_mse: 0.0044 - Press_loss: 7.7398 - Press_mse: 7.7420 - loss: 7.7464 
Epoch 7/1000
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - Mass_loss: 0.0033 - Mass_mse: 0.0032 - Press_loss: 7.0677 -

In [15]:
# Cria uma figura com subplots
fig = make_subplots(rows=1, cols=3, subplot_titles=("Mass Flow Rate vs Time", "Plenum Pressure vs Time"))

fig.add_trace(go.Scatter(x=list(range(1, len(history.history['Mass_loss']) + 1)), y=history.history['Mass_loss'], mode='lines', name='Mass Loss', line=dict(dash='solid')),
                  row=1, col=1)
fig.add_trace(go.Scatter(x=list(range(1, len(history.history['Press_loss']) + 1)), y=history.history['Press_loss'], mode='lines', name='Pressure Loss', line=dict(dash='solid')),
                  row=1, col=1)

mass, pressure = np.array(model.predict(x_test))
#max_mass = np.max(x_test[:,:,0])
#max_pressure = np.max(x_test[:,:,1])

# Gráfico da Mass Flow Rate
for i in range(nAlphas_Met, nAlphas):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(mass[:, i - nAlphas_Met]), mode='lines', name=f'Mass Flow Rate {i+1}', line=dict(dash='solid')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(massFlowrate[i]), mode='lines', name='Model Mass Flow Rate', line=dict(dash='dash', color='red')),
                  row=1, col=2)

# Gráfico da Plenum Pressure
for i in range(nAlphas_Met, nAlphas):
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(pressure[:, i - timestep]), mode='lines', name=f'Plenum Pressure {i+1}', line=dict(dash='solid')),
                  row=1, col=3)
    fig.add_trace(go.Scatter(x=interval[i], y=np.squeeze(PlenumPressure[i]), mode='lines', name= 'Model Plenum Pressure', line=dict(dash='dash', color='red')),
                  row=1, col=3)

# Atualiza layout
fig.update_layout(
    title='Mass Flow Rate and Plenum Pressure Over Time',
    xaxis_title='Time',
    yaxis_title='Value',
    template='plotly',
)

# Mostra a figura
fig.show()

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 639us/step
